In [1]:
from TPMetrics import TPMetrics
import configparser
import os
import pandas as pd
import numpy as np
import re
from itertools import repeat

In [2]:
tpm = TPMetrics(r'/home/rafael/CNIC/Metabolomics/TurboPutativeCode/testing/std_test_TPMetric')
tpm.getClasses()
tpm.getCorrelations()

In [10]:
tpm.df.drop(tpm.i, axis=1)

,index,FeatureInfo_Name,Apex m/z,RT [min],Identifier,Adduct,mz Error (ppm),Molecular Weight,Name,Peptide,...,TP_max_corr1_pvalue,TP_max_corr1_score,TP_max_corr2_pvalue,TP_max_corr2_score,TP_all_corr1_avg,TP_sum_corr1_pvalue,TP_sum_corr1_score,TP_all_corr2_avg,TP_sum_corr2_pvalue,TP_sum_corr2_score
0,0,A0401,117.05551,0.397,165908 // 165911 // 165909 // 165910 // 165912...,M-H,2.0,118.063,Hydroxyisopentanoate // Hydroxypentanoate // M...,NaN,...,[1.0],[-0.0],[1.0],[-0.0],[0.0],[1.0],[-0.0],[0.0],[1.0],[-0.0]
1,1,A0401,117.05551,0.397,164055 // 26909 // 26910 // 26912 // 26916 // ...,M-H,2.0,118.063,LMSD{ FA 5:0;O },NaN,...,[1.0],[-0.0],[1.0],[-0.0],[0.0],[1.0],[-0.0],[0.044038141894847345],[0.7189278227014076],[0.006311313479515381]
2,2,A0401,117.05551,0.397,99659,M-H,2.0,118.063,Ethyl lactate,NaN,...,[1.0],[-0.0],NaN,NaN,[0.0],[1.0],[-0.0],NaN,NaN,NaN
3,3,A0401,117.05551,0.397,161706 // 48234 // 137848 // 33733 // 46579,M+HCOOH-H,3.0,72.0575,Epoxybutane // Tetrahydrofuran // Methylpropan...,NaN,...,[1.0],[-0.0],NaN,NaN,[0.0],[1.0],[-0.0],NaN,NaN,NaN
4,4,A0401,117.05551,0.397,94508 // 99389 // 85457,M+HCOOH-H,3.0,72.0575,Dimethyloxirane // Buten-ol,NaN,...,[1.0],[-0.0],NaN,NaN,[0.0],[1.0],[-0.0],NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,182,A0613,876.57315,17.623,4357 // 5177,M-H,3.0,877.5833,PS 43:6; LMSD{ PS 43:6 },NaN,...,[1.0],[-0.0],[1.0],[-0.0],[0.0],[1.0],[-0.0],[0.0],[1.0],[-0.0]
183,183,A0613,876.57315,17.623,101909 // 132699 // 111230 // 88468 // 149415 ...,M+HCOOH-H,3.0,831.5778,PE-NMe 42:7 // PC 40:7; LMSD{ PC 40:7 } // PC ...,NaN,...,[1.0],[-0.0],"[1.0, 1.0]","[-0.0, -0.0]",[0.0],[1.0],[-0.0],"[0.0, 0.0]","[1.0, 1.0]","[-0.0, -0.0]"
184,184,A0613,876.57315,17.623,3586 // 3914 // 3751 // 3805 // 1473 // 2569 /...,M-H+HCOONa,1.0,809.5935,PE 41:4; LMSD{ PE 41:4 } // PC 38:4; LMSD{ PC ...,NaN,...,[1.0],[-0.0],"[1.0, 1.0, 1.0]","[-0.0, -0.0, -0.0]",[0.0],[1.0],[-0.0],"[0.12914005773773074, 0.0, 0.0]","[0.28603374263751624, 1.0, 1.0]","[0.07019830530369453, -0.0, -0.0]"
185,185,A0350,283.24255,11.861,NaN,NaN,NaN,NaN,UNK,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
df = tpm.df.loc[:, ['index', tpm.s2a, tpm.s2s, tpm.s2n]].dropna().explode([tpm.s2a, tpm.s2s, tpm.s2n]).fillna(0)
df[tpm.s2avg] = [np.mean(np.abs(i)) for i in df[tpm.s2a]]

df[tpm.s2sp] = [
    1 if n==0 else (tpm.VcorrH0[int(n)-1,:]>s).sum()/tpm.VcorrH0.shape[1] 
    for s, n in zip(df[tpm.s2s].to_list(), df[tpm.s2n].to_list())
]

df.loc[df[tpm.s2sp] == 0, tpm.s2sp] = 1/tpm.VcorrH0.shape[1]

df[tpm.s2ss] = np.sqrt(df[tpm.s2n]) * np.abs(df[tpm.s2avg]) * (-np.log10(df[tpm.s2sp]))

tpm.df = pd.merge(
    tpm.df,
    df.groupby('index').agg(list).reset_index().loc[:, ['index', tpm.s2avg, tpm.s2sp, tpm.s2ss]],
    on='index',
    how='left'
)


In [34]:
tpm.df

,index,FeatureInfo_Name,Apex m/z,RT [min],C18_2000n,C18_2001n,C18_2002n,C18_2003n,C18_2004n,C18_2005n,...,TP_max_corr2,TP_sum_corr2,TP_n_corr2,TP_max_corr1_pvalue,TP_max_corr1_score,TP_max_corr2_pvalue,TP_max_corr2_score,TP_all_corr2_avg,TP_sum_corr2_pvalue,TP_sum_corr2_score
0,0,A0401,117.05551,0.397,1.002532,0.993012,0.993865,1.011227,1.006851,1.051055,...,[nan],[nan],[nan],[1.0],[-0.0],[1.0],[-0.0],[0.0],[1.0],[-0.0]
1,1,A0401,117.05551,0.397,1.002532,0.993012,0.993865,1.011227,1.006851,1.051055,...,[-0.044038141894847345],[0.044038141894847345],[1.0],[1.0],[-0.0],[1.0],[-0.0],[0.044038141894847345],[0.717560147748827],[0.006347732169906429]
2,2,A0401,117.05551,0.397,1.002532,0.993012,0.993865,1.011227,1.006851,1.051055,...,NaN,NaN,NaN,[1.0],[-0.0],NaN,NaN,NaN,NaN,NaN
3,3,A0401,117.05551,0.397,1.002532,0.993012,0.993865,1.011227,1.006851,1.051055,...,NaN,NaN,NaN,[1.0],[-0.0],NaN,NaN,NaN,NaN,NaN
4,4,A0401,117.05551,0.397,1.002532,0.993012,0.993865,1.011227,1.006851,1.051055,...,NaN,NaN,NaN,[1.0],[-0.0],NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,182,A0613,876.57315,17.623,1.065888,1.000654,0.983885,0.986109,0.995659,1.021085,...,[nan],[nan],[nan],[1.0],[-0.0],[1.0],[-0.0],[0.0],[1.0],[-0.0]
183,183,A0613,876.57315,17.623,1.065888,1.000654,0.983885,0.986109,0.995659,1.021085,...,"[nan, nan]","[nan, nan]","[nan, nan]",[1.0],[-0.0],"[1.0, 1.0]","[-0.0, -0.0]","[0.0, 0.0]","[1.0, 1.0]","[-0.0, -0.0]"
184,184,A0613,876.57315,17.623,1.065888,1.000654,0.983885,0.986109,0.995659,1.021085,...,"[-0.12914005773773074, nan, nan]","[0.12914005773773074, nan, nan]","[1.0, nan, nan]",[1.0],[-0.0],"[1.0, 1.0, 1.0]","[-0.0, -0.0, -0.0]","[0.12914005773773074, 0.0, 0.0]","[0.28315863032844163, 1.0, 1.0]","[0.07076490288837355, -0.0, -0.0]"
185,185,A0350,283.24255,11.861,0.985405,1.028198,1.050863,1.125451,1.021219,1.071146,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
print(4*3**(1/2))
print(4*2**(1/2))

print(4*3**(1))
print(4*2**(1))

[4*i**(1/2) for i in range(10)]

6.928203230275509
5.656854249492381
12
8


[0.0,
 4.0,
 5.656854249492381,
 6.928203230275509,
 8.0,
 8.94427190999916,
 9.797958971132712,
 10.583005244258363,
 11.313708498984761,
 12.0]

In [22]:
df

,index,TP_all_corr2,TP_sum_corr2,TP_n_corr2,TP_all_corr2_avg,TP_sum_corr2_pvalue,TP_sum_corr2_score
0,0,0,0.000000,0.0,0.000000,1.000000,-0.000000
1,1,[-0.044038141894847345],0.044038,1.0,0.044038,0.717560,0.006348
5,5,[-0.044038141894847345],0.044038,1.0,0.044038,0.717560,0.006348
14,14,0,0.000000,0.0,0.000000,1.000000,-0.000000
15,15,0,0.000000,0.0,0.000000,1.000000,-0.000000
...,...,...,...,...,...,...,...
183,183,0,0.000000,0.0,0.000000,1.000000,-0.000000
183,183,0,0.000000,0.0,0.000000,1.000000,-0.000000
184,184,[-0.12914005773773074],0.129140,1.0,0.129140,0.283159,0.070765
184,184,0,0.000000,0.0,0.000000,1.000000,-0.000000


In [5]:
df = tpm.df.drop(tpm.i, axis=1)

In [11]:
df = tpm.df.loc[:, ['index', tpm.s1m]].dropna().explode(tpm.s1m).fillna(0)
df[tpm.s1mp] = [(tpm.VcorrH0[0, :]>i).sum()/tpm.VcorrH0.shape[1] for i in df[tpm.s1m].to_list()]
df.loc[df[tpm.s1mp] == 0, tpm.s1mp] = 1/tpm.VcorrH0.shape[1]
df[tpm.s1ms] = np.abs(df[tpm.s1m]) * (-np.log10(df[tpm.s1mp]))

tpm.df = pd.merge(
    tpm.df,
    df.groupby('index').agg(list).reset_index().loc[:, ['index', tpm.s1mp, tpm.s1ms]],
    on='index',
    how='left'
)

,index,FeatureInfo_Name,Apex m/z,RT [min],C18_2000n,C18_2001n,C18_2002n,C18_2003n,C18_2004n,C18_2005n,...,TP_all_corr1,TP_max_corr1,TP_sum_corr1,TP_n_corr1,TP_all_corr2,TP_max_corr2,TP_sum_corr2,TP_n_corr2,TP_max_corr1_pvalue,TP_max_corr1_score
0,0,A0401,117.05551,0.397,1.002532,0.993012,0.993865,1.011227,1.006851,1.051055,...,[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[1.0],[-0.0]
1,1,A0401,117.05551,0.397,1.002532,0.993012,0.993865,1.011227,1.006851,1.051055,...,[nan],[nan],[nan],[nan],[[-0.044038141894847345]],[-0.044038141894847345],[0.044038141894847345],[1.0],[1.0],[-0.0]
2,2,A0401,117.05551,0.397,1.002532,0.993012,0.993865,1.011227,1.006851,1.051055,...,[nan],[nan],[nan],[nan],NaN,NaN,NaN,NaN,[1.0],[-0.0]
3,3,A0401,117.05551,0.397,1.002532,0.993012,0.993865,1.011227,1.006851,1.051055,...,[nan],[nan],[nan],[nan],NaN,NaN,NaN,NaN,[1.0],[-0.0]
4,4,A0401,117.05551,0.397,1.002532,0.993012,0.993865,1.011227,1.006851,1.051055,...,[nan],[nan],[nan],[nan],NaN,NaN,NaN,NaN,[1.0],[-0.0]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,182,A0613,876.57315,17.623,1.065888,1.000654,0.983885,0.986109,0.995659,1.021085,...,[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[1.0],[-0.0]
183,183,A0613,876.57315,17.623,1.065888,1.000654,0.983885,0.986109,0.995659,1.021085,...,[nan],[nan],[nan],[nan],"[nan, nan]","[nan, nan]","[nan, nan]","[nan, nan]",[1.0],[-0.0]
184,184,A0613,876.57315,17.623,1.065888,1.000654,0.983885,0.986109,0.995659,1.021085,...,[nan],[nan],[nan],[nan],"[[-0.12914005773773074], nan, nan]","[-0.12914005773773074, nan, nan]","[0.12914005773773074, nan, nan]","[1.0, nan, nan]",[1.0],[-0.0]
185,185,A0350,283.24255,11.861,0.985405,1.028198,1.050863,1.125451,1.021219,1.071146,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
tpm.df.drop(tpm.i, axis=1)

,index,FeatureInfo_Name,Apex m/z,RT [min],Identifier,Adduct,mz Error (ppm),Molecular Weight,Name,Peptide,...,Formula,TP_Class,TP_all_corr1,TP_max_corr1,TP_sum_corr1,TP_n_corr1,TP_all_corr2,TP_max_corr2,TP_sum_corr2,TP_n_corr2
0,0,A0401,117.05551,0.397,165908 // 165911 // 165909 // 165910 // 165912...,M-H,2.0,118.063,Hydroxyisopentanoate // Hydroxypentanoate // M...,NaN,...,C5H10O3,WE,[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan]
1,1,A0401,117.05551,0.397,164055 // 26909 // 26910 // 26912 // 26916 // ...,M-H,2.0,118.063,LMSD{ FA 5:0;O },NaN,...,C5H10O3,FA,[nan],[nan],[nan],[nan],[[-0.044038141894847345]],[-0.044038141894847345],[0.044038141894847345],[1.0]
2,2,A0401,117.05551,0.397,99659,M-H,2.0,118.063,Ethyl lactate,NaN,...,C5H10O3,NaN,[nan],[nan],[nan],[nan],NaN,NaN,NaN,NaN
3,3,A0401,117.05551,0.397,161706 // 48234 // 137848 // 33733 // 46579,M+HCOOH-H,3.0,72.0575,Epoxybutane // Tetrahydrofuran // Methylpropan...,NaN,...,C4H8O,NaN,[nan],[nan],[nan],[nan],NaN,NaN,NaN,NaN
4,4,A0401,117.05551,0.397,94508 // 99389 // 85457,M+HCOOH-H,3.0,72.0575,Dimethyloxirane // Buten-ol,NaN,...,C4H8O,NaN,[nan],[nan],[nan],[nan],NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,182,A0613,876.57315,17.623,4357 // 5177,M-H,3.0,877.5833,PS 43:6; LMSD{ PS 43:6 },NaN,...,C49H84NO10P,PS,[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan]
183,183,A0613,876.57315,17.623,101909 // 132699 // 111230 // 88468 // 149415 ...,M+HCOOH-H,3.0,831.5778,PE-NMe 42:7 // PC 40:7; LMSD{ PC 40:7 } // PC ...,NaN,...,C48H82NO8P,PC // PE-NMe,[nan],[nan],[nan],[nan],"[nan, nan]","[nan, nan]","[nan, nan]","[nan, nan]"
184,184,A0613,876.57315,17.623,3586 // 3914 // 3751 // 3805 // 1473 // 2569 /...,M-H+HCOONa,1.0,809.5935,PE 41:4; LMSD{ PE 41:4 } // PC 38:4; LMSD{ PC ...,NaN,...,C46H84NO8P,PE // PC // PE-NMe,[nan],[nan],[nan],[nan],"[[-0.12914005773773074], nan, nan]","[-0.12914005773773074, nan, nan]","[0.12914005773773074, nan, nan]","[1.0, nan, nan]"
185,185,A0350,283.24255,11.861,NaN,NaN,NaN,NaN,UNK,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
'''
Incluir dos metodos; uno para el calculo de cada tipo de score
a partir de las columnas max, sum y n.
Cada metodo te genera una columna con lista de scores (un score para cada valor que tenía, ya iremos borrando)

Sumar pareja de scores en W y en K (el de K ponderado por aducto)

Coger maximo score en cada caso y sumar (restando el error de ppm, midiendo por el maximo)
'''

In [3]:
tpm.VcorrH0

array([[0.2536436 , 0.02839647, 0.07591637, ..., 0.17697489, 0.19261657,
        0.15650424],
       [0.45133409, 0.41427697, 0.14689878, ..., 0.07967807, 0.28273992,
        0.29302773],
       [0.29783921, 0.10489021, 0.38507567, ..., 0.2540985 , 0.14908582,
        0.1204619 ]])

In [7]:
# Get maximum number of correlations
maxn = tpm.df.loc[:, tpm.s1n].dropna().to_list()+tpm.df.loc[:, tpm.s2n].dropna().to_list()
maxn = [[j for j in i if not pd.isna(j)] for i in maxn]
maxn = int(max([max(i) for i in maxn if i!=[]]))

# Get number of shuffling
arr = tpm.df.loc[:, tpm.i].drop_duplicates().to_numpy()
T = 100000 # Number of correlation values
K = (2 * T)/(arr.shape[0]**2-arr.shape[0])
K = int(np.ceil(K))

In [11]:
#np.reshape(arrl[0], )
#arrl[0].reshape(arrl[0].shape[0]*arrl[0].shape[1])

arrl = [arr.copy().reshape(arr.shape[0]*arr.shape[1]) for i in range(K)]
_ = [np.random.shuffle(i) for i in arrl]
arrl = [i.reshape(arr.shape) for i in arrl]
Mcorr = [pd.DataFrame(i).T.corr(method='pearson').to_numpy() for i in arrl]

idx = np.triu_indices(arr.shape[0], 1)
Vcorr = np.concatenate([M[idx[0], idx[1]] for M in Mcorr])

In [12]:
VcorrH0 = [np.abs(Vcorr)]
for n in range(2, maxn+1):
    Vcorrl = [Vcorr.copy() for i in range(n)]
    _ = [np.random.shuffle(i) for i in Vcorrl]
    VcorrH0.append(np.sum(np.abs(Vcorrl), axis=0))

In [22]:
np.array([Vcorr]).shape

(1, 100170)

In [13]:
l = pd.DataFrame(VcorrH0).T

In [16]:
ll = tpm.df.drop(tpm.i, axis=1)

In [67]:
l1 = pd.DataFrame(l).T

In [62]:
l1 = Mcorr[0]
l2 = Mcorr[1]

In [55]:
pd.DataFrame(np.array(arrl).T)

,0,1,2,3,4,5,6
0,0.985708,0.987089,1.051084,1.013830,0.979157,1.122918,1.025794
1,1.018888,1.024418,1.027438,1.004965,1.040798,1.044985,1.250881
2,0.953812,0.989872,0.998790,1.012593,1.043622,0.947874,1.022043
3,1.055082,0.993067,1.076433,1.039065,0.991791,1.096830,1.051702
4,1.037905,1.011414,0.961251,0.993783,0.983500,0.969699,1.068035
...,...,...,...,...,...,...,...
3775,1.026872,1.057145,1.064493,1.039834,0.930957,1.074419,1.085407
3776,0.997681,1.035311,1.055509,1.029101,1.076310,1.244329,0.997283
3777,0.976868,1.025237,0.944177,0.948649,1.028277,1.037647,1.010829
3778,1.040091,1.048149,1.107618,1.041008,1.071449,1.117643,1.003626


In [24]:
arrl = [arr.copy() for i in range(K)]
_ = [[np.random.shuffle(i) for i in arri] for arri in arrl]
_ = [[np.random.shuffle(i) for i in arri.T] for arri in arrl]

Mcorr = [pd.DataFrame(i).T.corr(method='spearman').to_numpy() for i in arrl]

In [19]:
idx = np.triu_indices(arr.shape[0], 1)
l = np.concatenate([M[idx[0], idx[1]] for M in Mcorr])

In [32]:
# Get maximum number of correlations
maxn = tpm.df.loc[:, tpm.s1n].dropna().to_list()+tpm.df.loc[:, tpm.s2n].dropna().to_list()
maxn = [[j for j in i if not pd.isna(j)] for i in maxn]
maxn = max([max(i) for i in maxn if i!=[]])

# Generate matrixes with shuffled intentities
arr = tpm.df.loc[:, tpm.i].drop_duplicates().to_numpy()
# _ = [np.random.shuffle(i) for i in arr]
# _ = [np.random.shuffle(i) for i in arr.T]


In [47]:
arrl = [arr.copy() for i in range(max(10, maxn))]
_ = [[np.random.shuffle(i) for i in arri] for arri in arrl]
_ = [[np.random.shuffle(i) for i in arri.T] for arri in arrl]

arrl = [pd.DataFrame(i).T.corr(method='spearman') for i in arrl]


In [83]:
idx = np.triu_indices(arrl[0].shape[0],1)
arrl[0][*idx]

SyntaxError: invalid syntax (1718028422.py, line 2)

In [58]:
arrl[0].shape

(54, 54)

In [59]:
(54**2-54)/2

1431.0

In [38]:
l1 = arrl[0]

In [39]:
l2 = arrl[2]

In [5]:
df = tpm.df.loc[:, [tpm.s1a, tpm.s2a]].dropna()
l1 = [[j for j in i if type(j) == list] for i in df.loc[:, tpm.s1a].to_list()]
l2 = [[j for j in i if type(j) == list] for i in df.loc[:, tpm.s2a].to_list()]

[i for i in l1+l2 if i != []]
#[i for i in tpm.df.loc[:, tpm.s2a].to_list()]

[[[0.9476511241361211]],
 [[0.9476511241361211]],
 [[0.8684279590587001]],
 [[0.9837634502668183]],
 [[0.8684279590587001]],
 [[0.9837634502668183]],
 [[0.6627066748315983]],
 [[0.6627066748315983]],
 [[-0.044038141894847345]],
 [[0.281847607383431]],
 [[0.281847607383431]],
 [[0.281847607383431]],
 [[0.281847607383431]],
 [[0.38773510628991337]],
 [[0.5962557956434258], [0.5962557956434258]],
 [[0.9508354474674132, 0.9958358848744642],
  [0.9508354474674132, 0.9958358848744642],
  [0.9508354474674132, 0.9958358848744642]],
 [[0.9979354387192721], [0.9979354387192721]],
 [[0.8547108739392879, 0.8733969031580789, 0.9966407138483072]],
 [[0.38773510628991337]],
 [[0.8547108739392879, 0.754737118362348, 0.8578602047064998]],
 [[0.5962557956434258, 0.9508354474674132, 0.9476511241361211],
  [0.5962557956434258, 0.9508354474674132, 0.9476511241361211],
  [0.9508354474674132, 0.9476511241361211]],
 [[0.9958358848744642, 0.9476511241361211],
  [0.9958358848744642, 0.9476511241361211],
  [0.99

In [7]:
df = tpm.df.drop(tpm.i, axis=1)

In [3]:
df = tpm.df.loc[:, ['index',tpm.m, tpm.rt, tpm.tpc]].copy()
df[tpm.tpc] = tpm.df[tpm.tpc].str.split(' // ')
df = df.explode(tpm.tpc, ignore_index=True).dropna(axis=0, subset=[tpm.tpc]).astype({tpm.tpc: str})

In [4]:
df

,index,Apex m/z,RT [min],TP_Class
0,0,117.05551,0.397,WE
1,1,117.05551,0.397,FA
5,5,129.05569,0.412,FA
14,14,223.13396,6.885,WE
15,15,223.13396,6.885,FA
...,...,...,...,...
232,183,876.57315,17.623,PC
233,183,876.57315,17.623,PE-NMe
234,184,876.57315,17.623,PE
235,184,876.57315,17.623,PC


In [5]:
# [(index, mz, rt, mw)]
dfl = list(zip(*[j for i,j in df.to_dict('list').items()]))

# [(index, [pair_index_1, pair_index_2])]
idx2p = [
    ([index, w],
    df.loc[
        np.logical_and.reduce((
            m != df.loc[:, tpm.m],
            rt-tpm.rt1 <= df.loc[:, tpm.rt],
            rt+tpm.rt1 >= df.loc[:, tpm.rt],
            w == df.loc[:, tpm.tpc]
        )),
        'index'
    ].to_list())
    for index, m, rt, w in dfl
]

# remove empty elements and replace indexes by correlation
idx2p = [
    (
        index_w, 
        tpm.corr.loc[tpm.idx2mz[index_w[0]], [tpm.idx2mz[p] for p in pair]].to_list()
    ) 
    for index_w, pair in idx2p if len(pair)>0
]

# add maximum and sum; build dataframe
idx2p = pd.DataFrame(
    [
        [*index_w, pair, np.max(pair), np.sum(pair)]
        for index_w, pair in idx2p
    ], 
    columns=['index', tpm.tpc, tpm.s1a, tpm.s1m, tpm.s1s]
)

df = pd.merge(
    df,
    idx2p,
    on=['index', tpm.tpc],
    how='left'
).groupby('index').agg(list).reset_index()



In [7]:
df

,index,Apex m/z,RT [min],TP_Class,TP_all_corr1,TP_max_corr1,TP_sum_corr1
0,0,[117.05551],[0.397],[WE],[nan],[nan],[nan]
1,1,[117.05551],[0.397],[FA],[[-0.044038141894847345]],[-0.044038141894847345],[-0.044038141894847345]
2,5,[129.05569],[0.412],[FA],[[-0.044038141894847345]],[-0.044038141894847345],[-0.044038141894847345]
3,14,[223.13396],[6.885],[WE],[nan],[nan],[nan]
4,15,[223.13396],[6.885],[FA],[nan],[nan],[nan]
...,...,...,...,...,...,...,...
85,180,[835.53121],[17.52],[PI],[nan],[nan],[nan]
86,181,[835.53121],[17.52],[PG],[nan],[nan],[nan]
87,182,[876.57315],[17.623],[PS],[nan],[nan],[nan]
88,183,"[876.57315, 876.57315]","[17.623, 17.623]","[PC, PE-NMe]","[nan, nan]","[nan, nan]","[nan, nan]"


In [14]:
tpm.df = pd.merge(
    tpm.df,
    df,
    on='index',
    how='left'
)

In [15]:
df = tpm.df

In [107]:
dict(list(zip(
    tpm.df.index.to_list(),
    tpm.df.loc[:, tpm.m].to_list()
)))

{0: 117.05551,
 1: 117.05551,
 2: 117.05551,
 3: 117.05551,
 4: 117.05551,
 5: 129.05569,
 6: 129.05569,
 7: 129.05569,
 8: 129.05569,
 9: 129.05569,
 10: 129.05569,
 11: 129.05569,
 12: 129.05569,
 13: 129.05569,
 14: 223.13396,
 15: 223.13396,
 16: 223.13396,
 17: 223.13396,
 18: 223.13396,
 19: 223.13396,
 20: 223.13396,
 21: 250.14504,
 22: 250.14504,
 23: 250.14504,
 24: 250.14504,
 25: 271.22773,
 26: 271.22773,
 27: 271.22773,
 28: 271.22773,
 29: 299.25876,
 30: 299.25876,
 31: 299.25876,
 32: 299.25876,
 33: 339.19957,
 34: 339.19957,
 35: 339.25358,
 36: 339.25358,
 37: 339.25358,
 38: 339.25358,
 39: 353.21164,
 40: 353.21164,
 41: 353.21164,
 42: 353.21164,
 43: 353.21164,
 44: 353.21164,
 45: 353.21164,
 46: 353.21164,
 47: 353.21164,
 48: 369.17404,
 49: 369.17404,
 50: 369.17404,
 51: 369.17404,
 52: 369.17404,
 53: 369.17404,
 54: 369.17404,
 55: 369.17404,
 56: 369.17404,
 57: 369.17404,
 58: 378.24131,
 59: 378.24131,
 60: 378.24131,
 61: 378.24131,
 62: 383.15342,
 6

In [89]:
arr = tpm.df.loc[:, tpm.i].drop_duplicates().to_numpy()
_ = [np.random.shuffle(i) for i in arr]
_ = [np.random.shuffle(i) for i in arr.T]

In [99]:
_ = [np.random.shuffle(i) for i in arr]
_ = [np.random.shuffle(i) for i in arr.T]

In [18]:
'''
La columna con los molecular weight puede tener varios valores.
Hemos hecho explode, y consideramos cada uno por separado.
El proceso es igual... Para cada fila calculamos un score. 
Ya luego unimos por index, tomamos el mw de mayor score (dejar rastro de todas las columnas...)
'''

In [20]:
tpm.df

,FeatureInfo_Name,Apex m/z,RT [min],C18_2000n,C18_2001n,C18_2002n,C18_2003n,C18_2004n,C18_2005n,C18_2006n,...,Name,Peptide,Halogenated,Plant,NaturalProduct,MDM,Drug,Food,Formula,TP_Class
0,A0401,117.05551,0.397,1.002532,0.993012,0.993865,1.011227,1.006851,1.051055,1.032227,...,Hydroxyisopentanoate // Hydroxypentanoate // M...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C5H10O3,[WE]
1,A0401,117.05551,0.397,1.002532,0.993012,0.993865,1.011227,1.006851,1.051055,1.032227,...,LMSD{ FA 5:0;O },NaN,NaN,NaN,NP,NaN,Drug,Food,C5H10O3,[FA]
2,A0401,117.05551,0.397,1.002532,0.993012,0.993865,1.011227,1.006851,1.051055,1.032227,...,Ethyl lactate,NaN,NaN,NaN,NP,NaN,NaN,Food,C5H10O3,NaN
3,A0401,117.05551,0.397,1.002532,0.993012,0.993865,1.011227,1.006851,1.051055,1.032227,...,Epoxybutane // Tetrahydrofuran // Methylpropan...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C4H8O,NaN
4,A0401,117.05551,0.397,1.002532,0.993012,0.993865,1.011227,1.006851,1.051055,1.032227,...,Dimethyloxirane // Buten-ol,NaN,NaN,NaN,NaN,NaN,NaN,Food,C4H8O,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,A0613,876.57315,17.623,1.065888,1.000654,0.983885,0.986109,0.995659,1.021085,1.017898,...,PS 43:6; LMSD{ PS 43:6 },NaN,NaN,NaN,NaN,NaN,NaN,NaN,C49H84NO10P,[PS]
183,A0613,876.57315,17.623,1.065888,1.000654,0.983885,0.986109,0.995659,1.021085,1.017898,...,PE-NMe 42:7 // PC 40:7; LMSD{ PC 40:7 } // PC ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C48H82NO8P,"[PC, PE-NMe]"
184,A0613,876.57315,17.623,1.065888,1.000654,0.983885,0.986109,0.995659,1.021085,1.017898,...,PE 41:4; LMSD{ PE 41:4 } // PC 38:4; LMSD{ PC ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C46H84NO8P,"[PE, PC, PE-NMe]"
185,A0350,283.24255,11.861,0.985405,1.028198,1.050863,1.125451,1.021219,1.071146,1.033516,...,UNK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
tpm.df.loc[:, [tpm.m, tpm.rt, tpm.w]].to_numpy()

array([[117.05551, 0.397, '118.063'],
       [117.05551, 0.397, '118.063'],
       [117.05551, 0.397, '118.063'],
       [117.05551, 0.397, '72.0575'],
       [117.05551, 0.397, '72.0575'],
       [129.05569, 0.412, '130.063'],
       [129.05569, 0.412, '130.063'],
       [129.05569, 0.412, '130.063'],
       [129.05569, 0.412, '130.063'],
       [129.05569, 0.412, '84.0575'],
       [129.05569, 0.412, '84.0575'],
       [129.05569, 0.412, '84.0575'],
       [129.05569, 0.412, '260.126'],
       [129.05569, 0.412, '260.1273'],
       [223.13396, 6.885, '224.1412 // 224.1413'],
       [223.13396, 6.885, '224.1412 // 224.1413'],
       [223.13396, 6.885, '224.1412'],
       [223.13396, 6.885, '178.1358'],
       [223.13396, 6.885, '178.1358'],
       [223.13396, 6.885, '448.2825'],
       [223.13396, 6.885, '448.2825'],
       [250.14504, 9.07, '251.1521'],
       [250.14504, 9.07, '205.1467'],
       [250.14504, 9.07, '205.1467'],
       [250.14504, 9.07, '502.3083'],
       [271.22773,

In [5]:
tpm.df.loc[:, tpm.i]

,C18_2000n,C18_2001n,C18_2002n,C18_2003n,C18_2004n,C18_2005n,C18_2006n,C18_2007n,C18_2008n,C18_2009n,...,C18_C2408n,C18_C2409n,C18_C2410n,C18_C2501n,C18_C2601n,C18_C2602n,C18_C2603n,C18_C2604n,C18_C2605n,C18_C2606_n
0,1.002532,0.993012,0.993865,1.011227,1.006851,1.051055,1.032227,1.048686,1.006775,0.988408,...,1.005406,1.020761,1.006399,0.986268,0.989338,1.002750,0.988328,0.995823,1.002352,0.984438
1,1.002532,0.993012,0.993865,1.011227,1.006851,1.051055,1.032227,1.048686,1.006775,0.988408,...,1.005406,1.020761,1.006399,0.986268,0.989338,1.002750,0.988328,0.995823,1.002352,0.984438
2,1.002532,0.993012,0.993865,1.011227,1.006851,1.051055,1.032227,1.048686,1.006775,0.988408,...,1.005406,1.020761,1.006399,0.986268,0.989338,1.002750,0.988328,0.995823,1.002352,0.984438
3,1.002532,0.993012,0.993865,1.011227,1.006851,1.051055,1.032227,1.048686,1.006775,0.988408,...,1.005406,1.020761,1.006399,0.986268,0.989338,1.002750,0.988328,0.995823,1.002352,0.984438
4,1.002532,0.993012,0.993865,1.011227,1.006851,1.051055,1.032227,1.048686,1.006775,0.988408,...,1.005406,1.020761,1.006399,0.986268,0.989338,1.002750,0.988328,0.995823,1.002352,0.984438
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,1.065888,1.000654,0.983885,0.986109,0.995659,1.021085,1.017898,0.947634,1.031227,0.966813,...,0.980776,1.008965,1.011285,1.025889,1.029636,0.999024,1.060736,1.024878,0.996800,1.010608
183,1.065888,1.000654,0.983885,0.986109,0.995659,1.021085,1.017898,0.947634,1.031227,0.966813,...,0.980776,1.008965,1.011285,1.025889,1.029636,0.999024,1.060736,1.024878,0.996800,1.010608
184,1.065888,1.000654,0.983885,0.986109,0.995659,1.021085,1.017898,0.947634,1.031227,0.966813,...,0.980776,1.008965,1.011285,1.025889,1.029636,0.999024,1.060736,1.024878,0.996800,1.010608
185,0.985405,1.028198,1.050863,1.125451,1.021219,1.071146,1.033516,1.123164,1.004874,1.042944,...,1.040258,1.041212,0.991252,1.063642,1.058241,1.079337,1.049453,1.055571,1.050036,1.015847


In [4]:
[i for i in tpm.df.columns if tpm.ipatt.search(i)]


['C18_2000n',
 'C18_2001n',
 'C18_2002n',
 'C18_2003n',
 'C18_2004n',
 'C18_2005n',
 'C18_2006n',
 'C18_2007n',
 'C18_2008n',
 'C18_2009n',
 'C18_2010n',
 'C18_2101n',
 'C18_2201n',
 'C18_2202n',
 'C18_2203n',
 'C18_2204n',
 'C18_2205n',
 'C18_2301n',
 'C18_2401n',
 'C18_2402n',
 'C18_2403n',
 'C18_2404n',
 'C18_2405n',
 'C18_2406n',
 'C18_2407n',
 'C18_2408n',
 'C18_2409n',
 'C18_2410n',
 'C18_2501n',
 'C18_2601n',
 'C18_2602n',
 'C18_2603n',
 'C18_2604n',
 'C18_2605n',
 'C18_C1901n',
 'C18_C2001n',
 'C18_C2002n',
 'C18_C2003n',
 'C18_C2004n',
 'C18_C2005n',
 'C18_C2006_n',
 'C18_C2007n',
 'C18_C2008n',
 'C18_C2009n',
 'C18_C2010n',
 'C18_C2101n',
 'C18_C2201n',
 'C18_C2202n',
 'C18_C2203n',
 'C18_C2204n',
 'C18_C2205n',
 'C18_C2206n',
 'C18_C2301n',
 'C18_C2401n',
 'C18_C2402n',
 'C18_C2403n',
 'C18_C2404n',
 'C18_C2405n',
 'C18_C2406n',
 'C18_C2407n',
 'C18_C2408n',
 'C18_C2409n',
 'C18_C2410n',
 'C18_C2501n',
 'C18_C2601n',
 'C18_C2602n',
 'C18_C2603n',
 'C18_C2604n',
 'C18_C2605n'

In [56]:
with open(r'/home/rafael/CNIC/Metabolomics/TurboPutativeCode/src/TurboPutative-2.0-built/TPProcesser/TPMetrics/data/Lipid_Class.tsv', 'r') as f:
    l = [i.strip() for i in f]
ldf = pd.DataFrame({'TP_Class': l})

In [22]:
df = pd.read_csv(r'/home/rafael/CNIC/Metabolomics/TurboPutativeCode/testing/std_test_TPMetric/4_TableMerger.tsv', sep='\t')

In [58]:
df = df.join(pd.merge(
    pd.DataFrame(
        {
            'TP_Class': [
                [re.sub(r'LMSD{ ([^}]+) }', r'\1', j).split(' ')[0] for j in i.split(' // ')]
                for i in df['Name'].to_list()
            ],
            'idx': range(df.shape[0])
        }
    ).explode('TP_Class'),
    ldf,
    on='TP_Class',
    how='inner'
).drop_duplicates().groupby('idx').agg(list))

In [61]:
df.columns

Index(['FeatureInfo_Name', 'Apex m/z', 'RT [min]', 'C18_2000n', 'C18_2001n',
       'C18_2002n', 'C18_2003n', 'C18_2004n', 'C18_2005n', 'C18_2006n',
       'C18_2007n', 'C18_2008n', 'C18_2009n', 'C18_2010n', 'C18_2101n',
       'C18_2201n', 'C18_2202n', 'C18_2203n', 'C18_2204n', 'C18_2205n',
       'C18_2301n', 'C18_2401n', 'C18_2402n', 'C18_2403n', 'C18_2404n',
       'C18_2405n', 'C18_2406n', 'C18_2407n', 'C18_2408n', 'C18_2409n',
       'C18_2410n', 'C18_2501n', 'C18_2601n', 'C18_2602n', 'C18_2603n',
       'C18_2604n', 'C18_2605n', 'C18_C1901n', 'C18_C2001n', 'C18_C2002n',
       'C18_C2003n', 'C18_C2004n', 'C18_C2005n', 'C18_C2006_n', 'C18_C2007n',
       'C18_C2008n', 'C18_C2009n', 'C18_C2010n', 'C18_C2101n', 'C18_C2201n',
       'C18_C2202n', 'C18_C2203n', 'C18_C2204n', 'C18_C2205n', 'C18_C2206n',
       'C18_C2301n', 'C18_C2401n', 'C18_C2402n', 'C18_C2403n', 'C18_C2404n',
       'C18_C2405n', 'C18_C2406n', 'C18_C2407n', 'C18_C2408n', 'C18_C2409n',
       'C18_C2410n', 'C18_C250